# Sampler

The Sampler primitive lets you more accurately contextualize counts. It takes a user circuit as an input and generates an error mitigated readout of quasi-probabilities. This enables you to more efficiently evaluate the possibility of multiple relevant data points in the context of destructive interference.

`qiskit-ibm-runtime` now provides a simple `IBMSampler` class which allows access to the Sampler primitive service.

Instantiate the `IBMRuntimeService` from one of the saved accounts like below.

In [1]:
from qiskit_ibm_runtime import IBMRuntimeService

# Save cloud account on disk.
# IBMRuntimeService.save_account(auth="cloud", token=<IBM Cloud API key>, instance=<IBM Cloud CRN> or <IBM Cloud service name>)

# Or save legacy account on disk.
# IBMRuntimeService.save_account(auth="legacy", token=<IBM Quantum API key>)

service = IBMRuntimeService()

Instantiate the `IBMSampler` class by (optionally) passing the `service` instance instantiated above and also an optional backend name or `IBMBackend` object. 

Note:
- If `service` parameter is not set, it defaults to `IBMRuntimeService()` which means it will use your saved `cloud` or `legacy` accounts (in that order).
- If `backend` parameter is not set, then a least busy backend is automatically selected (IBM Cloud only).

In [2]:
from qiskit_ibm_runtime import IBMSampler

sampler_factory = IBMSampler(service=service)

Instantiating `IBMSampler` returns a factory which can be used to open a session with the Sampler primitive service and pass an initial list of `circuits` and `parameters` (optional). Let's prepare the quantum circuits as shown below.

In [3]:
from qiskit import QuantumCircuit

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

print(bell.draw())

        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 


The quantum circuits can then be passed as lists to the `sampler_factory`, which instantiates and opens a session with the Sampler primitive service and returns a handle to the session, `sampler`. `sampler` is an instance of `Sampler` class in `qiskit-ibm-runtime` which implements the `BaseSampler` interface in `qiskit-terra`.

`sampler` can then be called multiple times to write different `circuit_indices` (a list of circuit indices) and `parameter_values` (an optional list of concrete parameters to be bound) to the session and read the calculated quasi-probabilities. The result is an instance of `SamplerResult` class in `qiskit-terra` and it contains quasi-probabilities and metadata.

In [4]:
with sampler_factory(circuits=[bell]*3) as sampler:
    result1 = sampler(circuit_indices=[0, 1, 2], shots=2000)
    print(result1)
    
    result2 = sampler(circuit_indices=[0, 1, 2], shots=3000)
    print(result2)
    
    result3 = sampler(circuit_indices=[0, 1, 2], shots=4000)
    print(result3)

SamplerResult(quasi_dists=[{'00': 0.5155, '11': 0.4845}, {'11': 0.5175, '00': 0.4825}, {'00': 0.4915, '11': 0.5085}], metadata=[{'header_metadata': None, 'shots': 2000}, {'header_metadata': None, 'shots': 2000}, {'header_metadata': None, 'shots': 2000}])
SamplerResult(quasi_dists=[{'11': 0.5043333333333333, '00': 0.49566666666666664}, {'00': 0.501, '11': 0.499}, {'00': 0.493, '11': 0.507}], metadata=[{'header_metadata': None, 'shots': 3000}, {'header_metadata': None, 'shots': 3000}, {'header_metadata': None, 'shots': 3000}])
SamplerResult(quasi_dists=[{'00': 0.495, '11': 0.505}, {'11': 0.50075, '00': 0.49925}, {'11': 0.49825, '00': 0.50175}], metadata=[{'header_metadata': None, 'shots': 4000}, {'header_metadata': None, 'shots': 4000}, {'header_metadata': None, 'shots': 4000}])


In [5]:
from qiskit.tools.jupyter import *

%qiskit_copyright